In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

from nltk import flatten

# 1 - Import Files

In [2]:
MODEL_PATH = './Data/GSP5_DEE.htm'
DLINK_PATH = './Data/GSP5_DEE_TagMapping.xlsx'

#NAME = MODEL_PATH[MODEL_PATH.rfind('/')+1:MODEL_PATH.rfind('.')]
NAME = 'GSP5_DEE'

## PACE

In [ ]:
# initialize
MODEL = list()
xmlns = '{http://www.shell.com/APC/QuantumLeap}'

# import model
tree = ET.parse(MODEL_PATH)
blocks = tree.find(xmlns + 'Models/' + xmlns + 'Model/' + xmlns + 'Blocks')
for b in blocks:
    b = b.find(xmlns + 'SisoRelationship')
    MODEL.append([b.find(xmlns + 'InputVariable').text,
                  b.find(xmlns + 'OutputVariable').text,
                  b.find(xmlns + 'ActiveElement/' + xmlns + 'TransferFunction').text,
                  b.find(xmlns + 'ActiveElement/' + xmlns + 'Gain').text,
                  str(float(b.find(xmlns + 'ActiveElement/' + xmlns + 'Tau1').text)/60),
                  str(float(b.find(xmlns + 'ActiveElement/' + xmlns + 'Tau2').text)/60),
                  str(float(b.find(xmlns + 'ActiveElement/' + xmlns + 'Beta').text)/60),
                  str(float(b.find(xmlns + 'ActiveElement/' + xmlns + 'Delay').text)/60)])
MODEL = pd.DataFrame(MODEL, columns=['InputVariable', 'OutputVariable', 'Order',
                                     'Gain', 'Tau 1', 'Tau 2', 'Beta', 'Delay'])
MODEL = MODEL.astype({'InputVariable': str, 'OutputVariable': str, 'Order': str, 'Gain': float,
                      'Tau 1': float, 'Tau 2': float, 'Beta': float, 'Delay': float})

In [ ]:
# import data link
dfInput  = pd.read_excel(DLINK_PATH, sheet_name = 'Input')
dfOutput = pd.read_excel(DLINK_PATH, sheet_name = 'Output')
dfLink   = pd.read_excel(DLINK_PATH, sheet_name = 'Datalink')

# create variable dictionary
MV  = dfInput.loc[(dfInput['Ind'] == 'I') & (dfInput['MV'] == True) ]['Name'].to_dict()
DV  = dfInput.loc[(dfInput['Ind'] == 'I') & (dfInput['MV'] == False)]['Name'].to_dict()
CV  = dfOutput.loc[dfOutput['CV'] == True ]['Name'].to_dict()
POV = dfOutput.loc[dfOutput['CV'] == False]['Name'].to_dict()

# create tag mapping for controller
MAP = dict()
MAP['Name'] = ['Controller']
MAP['Tag']  = ['']

try:
    MAP['Mode'] = [dfLink.loc[dfLink['Reference'] == 'ModeActual', 'Reference.1'].item()]
except ValueError:
    MAP['Mode'] = ['']

MAP['High'] = ['']
MAP['Low']  = ['']
MAP['SST']  = ['']
MAP['PredErr']  = ['']
MAP['OptMode']  = ['']
MAP['OptCoeff'] = ['']
MAP['MaxMove']  = ['']
MAP['WindUp']   = ['']

# create tag mapping for MV
for mv in MV.values():
    MAP['Name'].append(mv)
    MAP['Tag'].append(mv)

    # Mode
    try:
        MAP['Mode'].append(dfLink.loc[dfLink['Reference'] == f'Variables/{mv}/MV/ModeActual', 'Reference.1'].item())
    except ValueError:
        MAP['Mode'].append('')

    # High limit
    try:
        MAP['High'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{mv}/MV/HiLimit', 'Reference'].item())
    except ValueError:
        MAP['High'].append('')

    # Low limit
    try:
        MAP['Low'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{mv}/MV/LoLimit', 'Reference'].item())
    except ValueError:
        MAP['Low'].append('')

    # SST
    try:
        MAP['SST'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{mv}/MV/SteadyStateValue', 'Reference'].item())
    except ValueError:
        MAP['SST'].append('')

    MAP['PredErr'].append('')
    MAP['OptMode'].append('')
    MAP['OptCoeff'].append('')

    # Max move
    try:
        MAP['MaxMove'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{mv}/MV/MaxMoveSizeUp', 'Reference'].item())
    except ValueError:
        MAP['MaxMove'].append('')

    MAP['WindUp'].append('')

# create tag mapping for DV
for dv in DV.values():
    MAP['Name'].append(dv)
    MAP['Tag'].append(dv)

    # Mode
    try:
        MAP['Mode'].append(dfLink.loc[dfLink['Reference'] == f'Variables/{dv}/DV/ModeActual', 'Reference.1'].item())
    except ValueError:
        MAP['Mode'].append('')

    MAP['High'].append('')
    MAP['Low'].append('')
    MAP['SST'].append('')
    MAP['PredErr'].append('')
    MAP['OptMode'].append('')
    MAP['OptCoeff'].append('')
    MAP['MaxMove'].append('')
    MAP['WindUp'].append('')

# create tag mapping for CV
for cv in CV.values():
    MAP['Name'].append(cv)
    MAP['Tag'].append(cv)

    # Mode
    try:
        MAP['Mode'].append(dfLink.loc[dfLink['Reference'] == f'Variables/{cv}/CV/ModeActual', 'Reference.1'].item())
    except ValueError:
        MAP['Mode'].append('')

    # High limit
    try:
        MAP['High'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{cv}/CV/HiLimit', 'Reference'].item())
    except ValueError:
        MAP['High'].append('')

    # Low limit
    try:
        MAP['Low'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{cv}/CV/LoLimit', 'Reference'].item())
    except ValueError:
        MAP['Low'].append('')

    # SST
    try:
        MAP['SST'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{cv}/CV/SteadyStateValue', 'Reference'].item())
    except ValueError:
        MAP['SST'].append('')

    # PredErr
    try:
        MAP['PredErr'].append(dfLink.loc[dfLink['Reference'] == f'Variables/{cv}/POV/PredictedValue', 'Reference.1'].item())
    except ValueError:
        MAP['PredErr'].append('')

    MAP['OptMode'].append('')
    MAP['OptCoeff'].append('')
    MAP['MaxMove'].append('')
    MAP['WindUp'].append('')

# create tag mapping for POV
#for pov in POV.values():
#    MAP['Name'].append(pov)
#    MAP['Tag'].append(pov)
#
#    # Mode
#    try:
#        MAP['Mode'].append(dfLink.loc[dfLink['Reference'] == f'Variables/{pov}/POV/ModeActual', 'Reference.1'].item())
#    except ValueError:
#        MAP['Mode'].append('')
#
#    # High limit
#    try:
#        MAP['High'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{pov}/POV/HiLimit', 'Reference'].item())
#    except ValueError:
#        MAP['High'].append('')
#
#    # Low limit
#    try:
#        MAP['Low'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{pov}/POV/LoLimit', 'Reference'].item())
#    except ValueError:
#        MAP['Low'].append('')
#
#    # SST
#    try:
#        MAP['SST'].append(dfLink.loc[dfLink['Reference.1'] == f'Variables/{pov}/POV/SteadyStateValue', 'Reference'].item())
#    except ValueError:
#        MAP['SST'].append('')
#
#    # PredErr
#    try:
#        MAP['PredErr'].append(dfLink.loc[dfLink['Reference'] == f'Variables/{pov}/POV/PredictedValue', 'Reference.1'].item())
#    except ValueError:
#        MAP['PredErr'].append('')
#
#    MAP['OptMode'].append('')
#    MAP['OptCoeff'].append('')
#    MAP['MaxMove'].append('')
#    MAP['WindUp'].append('')

MAP = pd.DataFrame.from_dict(MAP)

## SMOC

In [3]:
# import model
df_model  = pd.read_html(MODEL_PATH)[1]
df_model = df_model.rename(columns=df_model.iloc[0]).drop(df_model.index[0])
df_model = df_model.dropna(subset=['Model'])

# create variable dictionary
MV  = df_model[df_model['Model'].str.contains('MV')]['Block Name'].to_dict()
DV  = df_model[df_model['Model'].str.contains('DV')]['Block Name'].to_dict()
CV  = dict()
POV = df_model[df_model['Model'].str.contains('Output')]['Block Name'].to_dict()

# remove economic function variables
POV = {i:pov for i, pov in POV.items() if 'min' not in pov.lower()}
POV = {i:pov for i, pov in POV.items() if 'max' not in pov.lower()}
POV = {i:pov for i, pov in POV.items() if '-ll' not in pov.lower()}
POV = {i:pov for i, pov in POV.items() if '-hh' not in pov.lower()}
POV = {i:pov for i, pov in POV.items() if '-tar' not in pov.lower()}

In [4]:
# import data link
df_mv  = pd.read_excel(DLINK_PATH, sheet_name = 'MV Details')
df_dv  = pd.read_excel(DLINK_PATH, sheet_name = 'DV Details')
df_cv  = pd.read_excel(DLINK_PATH, sheet_name = 'CV Details')
df_pov = pd.read_excel(DLINK_PATH, sheet_name = 'POV Details')
df_sub = pd.read_excel(DLINK_PATH, sheet_name = 'SubController')

# create mapping dataframe
MAP = dict()
MAP['Name'] = ['Controller']
MAP['Tag']  = ['']
MAP['Mode'] = [df_sub.loc[df_sub['Alias'] == 'Actual Status [t/v]'].iloc[:, 1].item()]
MAP['High'] = ['']
MAP['Low']  = ['']
MAP['SST']  = ['']
MAP['PredErr']  = ['']
MAP['OptMode']  = ['']
MAP['OptCoeff'] = ['']
MAP['MaxMove']  = ['']
MAP['WindUp']   = ['']

for mv in MV.values():
    sst = df_mv.loc[df_mv['Alias'] == 'Steady State Reachable Value [t/v]', mv].item()
        
    MAP['Name'].append(mv)
    MAP['Tag'].append(df_mv.loc[df_mv['Alias'] == 'Setpoint Readback Tag [t]', mv].item())
    MAP['Mode'].append(df_mv.loc[df_mv['Alias'] == 'Remote / Local Flag Tag [t]', mv].item())
    MAP['High'].append(df_mv.loc[df_mv['Alias'] == 'Setpoint High Limit [t/v]', mv].item())
    MAP['Low'].append(df_mv.loc[df_mv['Alias'] == 'Setpoint Low Limit [t/v]', mv].item())
    MAP['SST'].append(['' if sst==0 else sst][0])
    MAP['PredErr'].append('')
    MAP['OptMode'].append('')
    MAP['OptCoeff'].append('')
    MAP['MaxMove'].append(df_mv.loc[df_mv['Alias'] == 'Maximum Move Size [t/v]', mv].item())
    MAP['WindUp'].append('')

for dv in DV.values():
    MAP['Name'].append(dv)
    MAP['Tag'].append(df_dv.loc[df_dv['Alias'] == 'Measurement Tag [t]', dv].item())
    MAP['Mode'].append(df_dv.loc[df_dv['Alias'] == 'Disconnect Flag [t/v]', dv].item())
    MAP['High'].append('')
    MAP['Low'].append('')
    MAP['SST'].append('')
    MAP['PredErr'].append('')
    MAP['OptMode'].append('')
    MAP['OptCoeff'].append('')
    MAP['MaxMove'].append('')
    MAP['WindUp'].append('')

for i, pov in POV.copy().items():
    if pov in df_cv.columns:
        CV[i] = POV.pop(i)
        MAP['Name'].append(pov)
        MAP['Tag'].append(df_pov.loc[df_pov['Alias'] == 'Measurement Tag [t]', pov].item())
        MAP['Mode'].append(df_cv.loc[df_cv['Alias'] == 'Actual CV Status [t/v]', pov].item())
        MAP['High'].append(df_cv.loc[df_cv['Alias'] == 'SetRange High [t/v]', pov].item())
        MAP['Low'].append(df_cv.loc[df_cv['Alias'] == 'SetRange Low [t/v]', pov].item())
        MAP['SST'].append(df_cv.loc[df_cv['Alias'] == 'Steady State Reachable Value [t/v]', pov].item())
        MAP['PredErr'].append(df_cv.loc[df_cv['Alias'] == 'Calculated Value [t/v]', pov].item())
        MAP['OptMode'].append('')
        MAP['OptCoeff'].append('')
        MAP['MaxMove'].append('')
        MAP['WindUp'].append('')
#    else:
#        MAP['Name'].append(pov)
#        MAP['Tag'].append(df_pov.loc[df_pov['Alias'] == 'Measurement Tag [t]', pov].item())
#        MAP['Mode'].append('')
#        MAP['High'].append('')
#        MAP['Low'].append('')
#        MAP['SST'].append('')
#        MAP['PredErr'].append('')
#        MAP['OptMode'].append('')
#        MAP['OptCoeff'].append('')
#        MAP['MaxMove'].append('')
#        MAP['WindUp'].append('')

MAP = pd.DataFrame.from_dict(MAP)

In [5]:
# reset index
MV = {i: v for i, v in enumerate(MV.values())}
DV = {i: v for i, v in enumerate(DV.values())}
CV = {i: v for i, v in enumerate(CV.values())}

# create model dataframe
MODEL = df_model[~(df_model['Model'].str.contains('Input') | df_model['Model'].str.contains('Output'))].reset_index()
MODEL = MODEL[['Input', 'Output', 'Model', 'Gain', 'Tau 1', 'Tau 2', 'Beta', 'Dead Time']].fillna(0.0)
MODEL = MODEL.astype({'Input': str, 'Output': str, 'Model': str, 'Gain': float, 
                      'Tau 1': float, 'Tau 2': float, 'Beta': float, 'Dead Time': float})
MODEL.columns = ['InputVariable', 'OutputVariable', 'Order', 'Gain', 'Tau 1', 'Tau 2', 'Beta', 'Delay']

In [6]:
# print result
print('MV :\n', list(MV.values()))
print('\nDV :\n', list(DV.values()))
print('\nCV :\n', list(CV.values()))
print('\nPOV :\n', list(POV.values()))
print('\nMODEL :\n', MODEL)

MV :
 ['3503FICA012.SV', '3503FICA016.SV', '3503LICA016.MV', '3503PICA032.SV', '3503TICA058.SV', '3511FICA004.SV', '3519FICA002.SV']

DV :
 ['35011QICA007.PV', '3503FICA011.PV', '3503PICA033-DV.PV', '3503TI023.PV', '3508TI018.PV']

CV :
 ['3503LICA010.PV', '3503LICA016.PV', '3503PICA033.PV', '3503QI017C2H6.PV', '3503TI040.PV', '3503TI041.PV.PV', '3504QT004C2H6', '3519FICA002.PV', '3519QT002C3H8B.PV', '3519QT002CO2A.PV']

POV :
 ['TICA058']

MODEL :
          InputVariable     OutputVariable         Order       Gain   Tau 1  \
0     3503QI017C2H6.PV      3504QT004C2H6   First Order   2.000000  30.000   
1      TICA058 (Error)            First_1          Gain  -0.950000   0.000   
2       3519FICA002.SV     3519FICA002.PV   First Order   1.000000   1.000   
3       3519FICA002.SV     3503LICA016.PV   First Order  -0.036100  60.000   
4       3519FICA002.SV     3503PICA033.PV   First Order  -0.000073  25.670   
5       3511FICA004.SV   3519QT002CO2A.PV   First Order -10.000000   4.000   


In [7]:
# manual correct model variables
MODEL = MODEL.replace('3503TI041.PV.PV', '3503TI041.PV')
MODEL = MODEL.replace('FICA012 to LICA010', '3503FICA012.SV')

# 2 - Processing

In [8]:
# create bond
BOND = list(MODEL[['InputVariable', 'OutputVariable']].itertuples(index = True, name = None))

# initialize graph
GRAPH = dict()
for i in MV.values(): GRAPH[i] = []
for i in DV.values(): GRAPH[i] = []

# append graph with bond
def recursive(index, input, output, parent, mode):
    if mode == 'input':
        model = MODEL[MODEL['InputVariable'] == input]['OutputVariable']
    elif mode == 'output':
        model = MODEL[MODEL['InputVariable'] == output]['OutputVariable']
    
    if len(model) > 0:
        for m in zip(model.index, model):
            print(f'Possible Route: {m}')
            idx, val = m
            if mode == 'input':
                index = idx
            elif mode == 'output':
                if parent[-1] == input:
                    index = tuple(flatten(index, idx))
                else:
                    index.pop()
            
            if val in CV.values():
                GRAPH[input].append((index, val))
                print(f'*** GRAPH[{input}] = ({index}, {val})')
                
                parent.append(input)
                recursive(index=index, input=input, output=val, parent=parent, mode='output')
            else:
                parent.append(input)
                recursive(index=index, input=input, output=val, parent=parent, mode='output')

for i, u, v in BOND:
    print(f'\nBond: i={i} u={u} v={v}')
    if u in MV.values() or u in DV.values():
        if v in CV.values():
            recursive(index=i, input=u, output=v, parent=[u], mode='input')
        else:
            recursive(index=i, input=u, output=v, parent=[u], mode='input')


Bond: i=0 u=3503QI017C2H6.PV v=3504QT004C2H6

Bond: i=1 u=TICA058 (Error) v=First_1

Bond: i=2 u=3519FICA002.SV v=3519FICA002.PV
Possible Route: (2, '3519FICA002.PV')
*** GRAPH[3519FICA002.SV] = (2, 3519FICA002.PV)
Possible Route: (3, '3503LICA016.PV')
*** GRAPH[3519FICA002.SV] = (3, 3503LICA016.PV)
Possible Route: (4, '3503PICA033.PV')
*** GRAPH[3519FICA002.SV] = (4, 3503PICA033.PV)

Bond: i=3 u=3519FICA002.SV v=3503LICA016.PV
Possible Route: (2, '3519FICA002.PV')
*** GRAPH[3519FICA002.SV] = (2, 3519FICA002.PV)
Possible Route: (3, '3503LICA016.PV')
*** GRAPH[3519FICA002.SV] = (3, 3503LICA016.PV)
Possible Route: (4, '3503PICA033.PV')
*** GRAPH[3519FICA002.SV] = (4, 3503PICA033.PV)

Bond: i=4 u=3519FICA002.SV v=3503PICA033.PV
Possible Route: (2, '3519FICA002.PV')
*** GRAPH[3519FICA002.SV] = (2, 3519FICA002.PV)
Possible Route: (3, '3503LICA016.PV')
*** GRAPH[3519FICA002.SV] = (3, 3503LICA016.PV)
Possible Route: (4, '3503PICA033.PV')
*** GRAPH[3519FICA002.SV] = (4, 3503PICA033.PV)

Bond:

In [9]:
BOND

[(0, '3503QI017C2H6.PV', '3504QT004C2H6'),
 (1, 'TICA058 (Error)', 'First_1'),
 (2, '3519FICA002.SV', '3519FICA002.PV'),
 (3, '3519FICA002.SV', '3503LICA016.PV'),
 (4, '3519FICA002.SV', '3503PICA033.PV'),
 (5, '3511FICA004.SV', '3519QT002CO2A.PV'),
 (6, '3503FICA011.PV', '3503PICA033.PV'),
 (7, '3503FICA011.PV', '3503TI040.PV'),
 (8, '3503FICA011.PV', '3503TI041.PV'),
 (9, '3503FICA012.SV', 'Ramp LICA010.PV'),
 (10, '3503FICA016.SV', '3503TI041.PV'),
 (11, '3503TICA058.SV', 'TICA058'),
 (12, '3503LICA016.MV', '3503LICA016.PV'),
 (13, '3503PICA032.SV', '3503LICA016.PV'),
 (14, '3503PICA032.SV', '3503PICA033.PV'),
 (15, '3503PICA033-DV.PV', '3503QI017C2H6.PV'),
 (16, '3503PICA033-DV.PV', '3503TI041.PV'),
 (17, '35011QICA007.PV', '3519QT002CO2A.PV'),
 (18, '3503FICA012.SV', '3503LICA010.PV'),
 (19, '3508TI018.PV', '3503TI040.PV'),
 (20, '3503TI023.PV', '3503TI041.PV'),
 (21, '3503TI040.PV', '3503QI017C2H6.PV'),
 (22, '3503TI041.PV', '3519QT002C3H8B.PV'),
 (23, 'TICA058', '3503TI040.PV')]

In [10]:
GRAPH

{'3503FICA012.SV': [(18, '3503LICA010.PV'), (18, '3503LICA010.PV')],
 '3503FICA016.SV': [((10, 22), '3519QT002C3H8B.PV')],
 '3503LICA016.MV': [(12, '3503LICA016.PV')],
 '3503PICA032.SV': [(13, '3503LICA016.PV'),
  (14, '3503PICA033.PV'),
  (13, '3503LICA016.PV'),
  (14, '3503PICA033.PV')],
 '3503TICA058.SV': [((11, 23), '3503TI040.PV'),
  ((11, 23, 21), '3503QI017C2H6.PV'),
  ((11, 23, 21, 0), '3504QT004C2H6')],
 '3511FICA004.SV': [(5, '3519QT002CO2A.PV')],
 '3519FICA002.SV': [(2, '3519FICA002.PV'),
  (3, '3503LICA016.PV'),
  (4, '3503PICA033.PV'),
  (2, '3519FICA002.PV'),
  (3, '3503LICA016.PV'),
  (4, '3503PICA033.PV'),
  (2, '3519FICA002.PV'),
  (3, '3503LICA016.PV'),
  (4, '3503PICA033.PV')],
 '35011QICA007.PV': [(17, '3519QT002CO2A.PV')],
 '3503FICA011.PV': [(6, '3503PICA033.PV'),
  (7, '3503TI040.PV'),
  ((7, 21), '3503QI017C2H6.PV'),
  ((7, 21, 0), '3504QT004C2H6'),
  ((8, 22), '3519QT002C3H8B.PV'),
  (6, '3503PICA033.PV'),
  (7, '3503TI040.PV'),
  ((7, 21), '3503QI017C2H6.PV'),

In [11]:
# create MPC class
class MPC:
    def __init__(self, input='', output='', idx=None, model=None):
        self.Input = input
        self.Output = output
        
        if idx is not None:
            self.Gain = float(model.iloc[idx]['Gain'])
            self.Delay = float(model.iloc[idx]['Delay'])
            if model.iloc[idx]['Order'] == 'Gain':
                self.Num = [1]
                self.Den = [1]
            elif model.iloc[idx]['Order'] == 'FirstOrder' or model.iloc[idx]['Order'] == 'First Order':
                self.Num = [1]
                self.Den = [1, float(model.iloc[idx]['Tau 1'])]
            elif model.iloc[idx]['Order'] == 'Ramp':
                self.Num = [1]
                self.Den = [0, 1, float(model.iloc[idx]['Tau 1'])]
            elif model.iloc[idx]['Order'] == 'FirstOrderRamp':
                self.Num = [1, float(model.iloc[idx]['Beta'])]
                self.Den = [0, float(model.iloc[idx]['Tau 1']), float(model.iloc[idx]['Tau 2'])]
            elif model.iloc[idx]['Order'] == 'SecondOrderBeta' or model.iloc[idx]['Order'] == 'Second Order':
                self.Num = [1, float(model.iloc[idx]['Beta'])]
                self.Den = [1, float(model.iloc[idx]['Tau 1']), float(model.iloc[idx]['Tau 2'])]
            elif model.iloc[idx]['Order'] == 'ZeroGainSecondOrder' or model.iloc[idx]['Order'] == '2nd Order Zero Gain':
                self.Num = [0, float(model.iloc[idx]['Beta'])]
                self.Den = [1, float(model.iloc[idx]['Tau 1']), float(model.iloc[idx]['Tau 2'])]
        else:
            self.Gain  = 0
            self.Delay = 0
            self.Num   = []
            self.Den   = []
    
# create MPC multiplication function
def multMPC(mdl1, mdl2):
    result = MPC()
    result.Input = mdl1.Input
    result.Output = mdl2.Output
    
    # Gain
    result.Gain  = mdl1.Gain * mdl2.Gain
    
    # Delay
    result.Delay = mdl1.Delay + mdl2.Delay
    
    # Numerator
    for i in range(len(mdl1.Num) + len(mdl2.Num) - 1):
        result.Num.append(0)
    for i1 in range(len(mdl1.Num)):
        for i2 in range(len(mdl2.Num)):
            result.Num[i1+i2] = result.Num[i1+i2] + mdl1.Num[i1]*mdl2.Num[i2]
    
    # Denominator
    for i in range(len(mdl1.Den) + len(mdl2.Den) - 1):
        result.Den.append(0)
    for i1 in range(len(mdl1.Den)):
        for i2 in range(len(mdl2.Den)):
            result.Den[i1+i2] = result.Den[i1+i2] + mdl1.Den[i1]*mdl2.Den[i2]
    
    return result

In [12]:
# calculate final model
FINALMODEL = list()
for input, outputs in GRAPH.items():
    for o in outputs:
        index = o[0]
        output = o[1]
        if type(index) is int:
            fmodel = MPC(input=input, output=output, idx=index, model=MODEL)
        elif type(index) is tuple:
            for c, i in enumerate(index):
                if c == 0:
                    omodel = MPC(input=input, idx=i, model=MODEL)
                elif c > 0:
                    fmodel = multMPC(omodel, MPC(output=output, idx=i, model=MODEL))
                    omodel = fmodel
        FINALMODEL.append((fmodel.Input, fmodel.Output, fmodel.Gain, fmodel.Delay, fmodel.Num, fmodel.Den))

In [13]:
def removeDuplicate(inList):
    result = list()
    for i in inList:
        if i not in result:
            result.append(i)
    return result

FINALMODEL = removeDuplicate(FINALMODEL)
FINALMODEL

[('3503FICA012.SV', '3503LICA010.PV', 0.016667, 0.0, [1], [0, 1, 0.0]),
 ('3503FICA016.SV',
  '3519QT002C3H8B.PV',
  -0.0194,
  16.549999999999997,
  [1],
  [1, 17.43, 68.9022]),
 ('3503LICA016.MV', '3503LICA016.PV', -0.001, 0.0, [1], [1, 1.0]),
 ('3503PICA032.SV', '3503LICA016.PV', -0.001, 0.0, [1], [1, 1.0]),
 ('3503PICA032.SV', '3503PICA033.PV', 4.329, 0.041, [1], [1, 43.76]),
 ('3503TICA058.SV', '3503TI040.PV', 1.0, 3.865, [1], [1, 5.943, 4.943]),
 ('3503TICA058.SV',
  '3503QI017C2H6.PV',
  -0.02212,
  13.605,
  [1],
  [1, 10.543, 32.2808, 22.737799999999996]),
 ('3503TICA058.SV',
  '3504QT004C2H6',
  -0.04424,
  73.605,
  [1],
  [1, 40.543, 348.57079999999996, 991.1618, 682.1339999999999]),
 ('3511FICA004.SV', '3519QT002CO2A.PV', -10.0, 3.0, [1], [1, 4.0]),
 ('3519FICA002.SV', '3519FICA002.PV', 1.0, 0.0, [1], [1, 1.0]),
 ('3519FICA002.SV', '3503LICA016.PV', -0.0361, 10.0, [1], [1, 60.0]),
 ('3519FICA002.SV', '3503PICA033.PV', -7.3e-05, 1.0, [1], [1, 25.67]),
 ('35011QICA007.PV', '

In [14]:
# verify model and variable list
def checkFinalModel(model, mv, dv, cv):
    mv_check, dv_check, cv_check = [], [], []
    for i, o, _, _, _, _ in model:
        for m in mv.values():
            if m == i:
                mv_check.append(i)
        for d in dv.values():
            if d == i:
                dv_check.append(i)
        for c in cv.values():
            if c == o:
                cv_check.append(o)
    
    mv_check = list(set(mv_check))
    dv_check = list(set(dv_check))
    cv_check = list(set(cv_check))
    
    print('MV:', sorted(mv_check) == sorted(list(mv.values())))
    print(list(mv.values()))
    print(mv_check)
    print('\nCV:', sorted(cv_check) == sorted(list(cv.values())))
    print(list(cv.values()))
    print(cv_check)
    print('\nDV:', sorted(dv_check) == sorted(list(dv.values())))
    print(list(dv.values()))
    print(dv_check)

checkFinalModel(FINALMODEL, MV, DV, CV)

MV: True
['3503FICA012.SV', '3503FICA016.SV', '3503LICA016.MV', '3503PICA032.SV', '3503TICA058.SV', '3511FICA004.SV', '3519FICA002.SV']
['3503FICA016.SV', '3519FICA002.SV', '3503TICA058.SV', '3511FICA004.SV', '3503LICA016.MV', '3503PICA032.SV', '3503FICA012.SV']

CV: False
['3503LICA010.PV', '3503LICA016.PV', '3503PICA033.PV', '3503QI017C2H6.PV', '3503TI040.PV', '3503TI041.PV.PV', '3504QT004C2H6', '3519FICA002.PV', '3519QT002C3H8B.PV', '3519QT002CO2A.PV']
['3503PICA033.PV', '3519FICA002.PV', '3519QT002C3H8B.PV', '3519QT002CO2A.PV', '3503QI017C2H6.PV', '3504QT004C2H6', '3503LICA010.PV', '3503LICA016.PV', '3503TI040.PV']

DV: True
['35011QICA007.PV', '3503FICA011.PV', '3503PICA033-DV.PV', '3503TI023.PV', '3508TI018.PV']
['35011QICA007.PV', '3508TI018.PV', '3503FICA011.PV', '3503PICA033-DV.PV', '3503TI023.PV']


# 3 - Generate Outputs

In [15]:
LO2_PATH = './Result/GSP5_DEE_Model.LO2'
MAP_PATH = './Result/GSP5_DEE_TagMap.csv'
CONFIG_PATH = './Result/GSP5_DEE_Configuration.xml'

In [16]:
def replaceName(name):
    return name.upper().replace('-', '').replace('.', '_')

In [17]:
##################
# Model LO2 File #
##################

# check max order
MaxOrder = 0
for mdl in FINALMODEL:
    if MaxOrder < len(mdl[5]):
        MaxOrder = len(mdl[5])

# header
ModelContent = 'Normal format: G(s) = Gain*[('
for o in range(MaxOrder-2, 0, -1):
    if o == 1:
        ModelContent += 'B1*s+'
    else:
        ModelContent += f'B{o}*s^{o}+'
ModelContent += 'B0)/('
for o in range(MaxOrder-1, 0, -1):
    if o == 1:
        ModelContent += 'A1*s+'
    else:
        ModelContent += f'A{o}*s^{o}+'
ModelContent += 'A0)]*exp(-Delay*s)\n\n'

# Detail each MV-CV
for i, mv in enumerate(MV.values()):
    for j, cv in enumerate(CV.values()):
        for mdl in FINALMODEL:
            if mdl[0] == mv and mdl[1] == cv:
                ModelContent += f'MV{i+1}: {replaceName(mv)}, CV{j+1}: {replaceName(cv)}\n'
                ModelContent += f'Gain = {round(mdl[2], 10)}, '
                for o in range(MaxOrder-2, -1, -1):
                    try:
                        ModelContent += f'B{o} = {round(mdl[4][o], 10)}, '
                    except:
                        ModelContent += f'B{o} = 0.0, '
                for o in range(MaxOrder-1, -1, -1):
                    try:
                        ModelContent += f'A{o} = {round(mdl[5][o], 10)}, '
                    except:
                        ModelContent += f'A{o} = 0.0, '
                ModelContent += f'Delay = {round(mdl[3], 10)}\n\n'

# Detail each DV-CV
for i, dv in enumerate(DV.values()):
    for j, cv in enumerate(CV.values()):
        for mdl in FINALMODEL:
            if mdl[0] == dv and mdl[1] == cv:
                ModelContent += f'MV{len(MV)+i+1}: {replaceName(dv)}, CV{j+1}: {replaceName(cv)}\n'
                ModelContent += f'Gain = {round(mdl[2], 10)}, '
                for o in range(MaxOrder-2, -1, -1):
                    try:
                        ModelContent += f'B{o} = {round(mdl[4][o], 10)}, '
                    except:
                        ModelContent += f'B{o} = 0.0, '
                for o in range(MaxOrder-1, -1, -1):
                    try:
                        ModelContent += f'A{o} = {round(mdl[5][o], 10)}, '
                    except:
                        ModelContent += f'A{o} = 0.0, '
                ModelContent += f'Delay = {round(mdl[3], 10)}\n\n'
                
# Write LO2 model file
with open(LO2_PATH, 'w') as f:
    f.write(ModelContent[:-2])

In [18]:
########################
# Tag Mapping CSV File #
########################

# initialize
TagMapping = list()
con = MAP[MAP['Name']=='Controller']
mv  = MAP[MAP['Name'].isin(MV.values())]
dv  = MAP[MAP['Name'].isin(DV.values())]
cv  = MAP[MAP['Name'].isin(CV.values())]

# controller
if (con['Mode'].item() != '') and (con['Mode'].item() != '0') and (type(con['Mode'].item()) != int):
    TagMapping.append([con['Mode'].item(), NAME + '.MODE'])
if (con['OptMode'].item() != '') and (con['OptMode'].item() != '0') and (type(con['OptMode'].item()) != int):
    TagMapping.append([con['OptMode'].item(), NAME + '.OPT'])

# MV
for idx in mv.index:
    if (mv['Tag'][idx] != '') and (mv['Tag'][idx] != '0') and (type(mv['Tag'][idx]) != int):
        TagMapping.append([mv['Tag'][idx], replaceName(mv['Name'][idx])])
    if (mv['Mode'][idx] != '') and (mv['Mode'][idx] != '0') and (type(mv['Mode'][idx]) != int):
        TagMapping.append([mv['Mode'][idx], replaceName(mv['Name'][idx]) + '.MODE'])
    if (mv['High'][idx] != '') and (mv['High'][idx] != '0') and (type(mv['High'][idx]) != int):
        TagMapping.append([mv['High'][idx], replaceName(mv['Name'][idx]) + '.HIGH'])
    if (mv['Low'][idx] != '') and (mv['Low'][idx] != '0') and (type(mv['Low'][idx]) != int):
        TagMapping.append([mv['Low'][idx], replaceName(mv['Name'][idx]) + '.LOW'])
    if (mv['SST'][idx] != '') and (mv['SST'][idx] != '0') and (type( mv['SST'][idx]) != int):
        TagMapping.append([mv['SST'][idx], replaceName(mv['Name'][idx]) + '.SST'])
    if (mv['OptMode'][idx] != '') and (mv['OptMode'][idx] != '0') and (type(mv['OptMode'][idx]) != int):
        TagMapping.append([mv['OptMode'][idx], replaceName(mv['Name'][idx]) + '.OPTMODE'])
    if (mv['OptCoeff'][idx] != '') and (mv['OptCoeff'][idx] != '0') and (type(mv['OptCoeff'][idx]) != int):
        TagMapping.append([mv['OptCoeff'][idx], replaceName(mv['Name'][idx]) + '.OPTCOEFF'])
    if (mv['MaxMove'][idx] != '') and (mv['MaxMove'][idx] != '0') and (type(mv['MaxMove'][idx]) != int):
        TagMapping.append([mv['MaxMove'][idx], replaceName(mv['Name'][idx]) + '.MAXUP'])
        TagMapping.append([mv['MaxMove'][idx], replaceName(mv['Name'][idx]) + '.MAXDN'])
    if (mv['WindUp'][idx] != '') and (mv['WindUp'][idx] != '0') and (type(mv['WindUp'][idx]) != int):
        TagMapping.append([mv['WindUp'][idx], replaceName(mv['Name'][idx]) + '.WNUP'])

# DV
for idx in dv.index:
    if (dv['Tag'][idx] != '') and (dv['Tag'][idx] != '0') and (type(dv['Tag'][idx]) != int):
        TagMapping.append([dv['Tag'][idx], replaceName(dv['Name'][idx])])
    if (dv['Mode'][idx] != '') and (dv['Mode'][idx] != '0') and (type(dv['Mode'][idx]) != int):
        TagMapping.append([dv['Mode'][idx], replaceName(dv['Name'][idx]) + '.MODE'])

# CV
for idx in cv.index:
    if (cv['Tag'][idx] != '') and (cv['Tag'][idx] != '0') and (type(cv['Tag'][idx]) != int):
        TagMapping.append([cv['Tag'][idx], replaceName(cv['Name'][idx])])
    if (cv['Mode'][idx] != '') and (cv['Mode'][idx] != '0') and (type(cv['Mode'][idx]) != int):
        TagMapping.append([cv['Mode'][idx], replaceName(cv['Name'][idx]) + '.MODE'])
    if (cv['High'][idx] != '') and (cv['High'][idx] != '0') and (type(cv['High'][idx]) != int):
        TagMapping.append([cv['High'][idx], replaceName(cv['Name'][idx]) + '.HIGH'])
    if (cv['Low'][idx] != '') and (cv['Low'][idx] != '0') and (type(cv['Low'][idx]) != int):
        TagMapping.append([cv['Low'][idx], replaceName(cv['Name'][idx]) + '.LOW'])
    if (cv['SST'][idx] != '') and (cv['SST'][idx] != '0') and (type(cv['SST'][idx]) != int):
        TagMapping.append([cv['SST'][idx], replaceName(cv['Name'][idx]) + '.SST'])
    if (cv['PredErr'][idx] != '') and (cv['PredErr'][idx] != '0') and (type(cv['PredErr'][idx]) != int):
        TagMapping.append([cv['PredErr'][idx], replaceName(cv['Name'][idx]) + '.PREDERR'])
    if (cv['OptMode'][idx] != '') and (cv['OptMode'][idx] != '0') and (type(cv['OptMode'][idx]) != int):
        TagMapping.append([cv['OptMode'][idx], replaceName(cv['Name'][idx]) + '.OPTMODE'])
    if (cv['OptCoeff'][idx] != '') and (cv['OptCoeff'][idx] != '0') and (type(cv['OptCoeff'][idx]) != int):
        TagMapping.append([cv['OptCoeff'][idx], replaceName(cv['Name'][idx]) + '.OPTCOEFF'])

TagMapping = pd.DataFrame(TagMapping)
TagMapping.to_csv(MAP_PATH, header = False, index = False)

TagMapping

,0,1
0,G5M3DEESCSTS.SW,GSP5_DEE.MODE
1,3503FICA012.SV,3503FICA012_SV
2,G5M3MV07.DT01,3503FICA012_SV.MODE
3,G5M3MV07.DT11,3503FICA012_SV.HIGH
4,G5M3MV07.NX11,3503FICA012_SV.LOW
...,...,...
115,G5M3CV05.NX01,3519QT002CO2A_PV.MODE
116,G5M3CV05.DT10,3519QT002CO2A_PV.HIGH
117,G5M3CV05.NX10,3519QT002CO2A_PV.LOW
118,G5M3CV05.DT14,3519QT002CO2A_PV.SST


In [19]:
##########################
# Configuration XML File #
##########################

def checkSource(vtype, vdata):
    blank, constant, tag = 0, 0, 0
    for v in vdata:
        if v == '':
            blank += 1
        elif type(v) == int:
            constant += 1
        elif v in TagMapping[0].to_list():
            tag += 1
    
    if (vtype == 'MV' and len(mv) == tag) or (vtype == 'DV' and len(dv) == tag) or (vtype == 'CV' and len(cv) == tag):
        return 'External'
    elif constant > 0 and tag > 0:
        return 'ExternalConfig'
    elif (vtype == 'MV' and len(mv) == blank) or (vtype == 'DV' and len(dv) == blank) or (vtype == 'CV' and len(cv) == blank):
        return 'NotUsed'

def appendTree(ctype, parent, tag, extra, content='', hierarchy={}):
    if parent is None and ctype == 'open':
        if tag not in hierarchy.keys():
            hierarchy[tag] = 0
        content += '<?xml version="1.0" encoding="utf-8"?>\n'
    else:
        if tag not in ConfigParent.keys():
            hierarchy[tag] = hierarchy[parent]+1
        content += '\t'*ConfigParent[tag]
    
    if ctype == 'close':
        content += f'</{tag}>\n'
        hierarchy.pop(tag, None)
    elif ctype == 'self-close' and extra is None:
        content += f'<{tag} />\n'
        hierarchy.pop(tag, None)
    elif ctype == 'self-close' and extra is not None:
        content += f'<{tag}'
        for k, v in extra.items():
            content += f' {k}="{v}"'
        content += ' />\n'
        hierarchy.pop(tag, None)
    elif ctype == 'open' and extra is None:
        content += f'<{tag}>\n'
    elif ctype == 'open' and extra is not None:
        content += f'<{tag}'
        for k, v in extra.items():
            content += f' {k}="{v}"'
        content += '>\n'
    
    return content, hierarchy

# header
ConfigContent, ConfigParent = appendTree('open', None, 'Configuration', {'xmlns':'http://schemas.matrikon.com/2010/08/CPM/GenericAPC.xsd'})

# controller
ConfigContent, ConfigParent = appendTree('open', 'Configuration', 'Controller', {'Name': NAME}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Controller', 'ExecutionInterval', {'Value': 60}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Controller', 'HP', {'Value': 180}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Controller', 'Mode', {'Source': 'External'}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Controller', 'OptMode', {'Source': 'NotUsed'}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Controller', 'OptCost', {'Source': 'NotUsed'}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('close', 'Configuration', 'Controller', None, ConfigContent, ConfigParent)

# MV
ConfigContent, ConfigParent = appendTree('open', 'Configuration', 'MVs', None, ConfigContent, ConfigParent)

ConfigContent, ConfigParent = appendTree('open', 'MVs', 'Defaults', None, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'Mode', {'Source': checkSource('MV', mv['Mode'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'High', {'Source': checkSource('MV', mv['High'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'Low', {'Source': checkSource('MV', mv['Low'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'HEnb', {'Source': 'Config', 'Value': 1}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'LEnb', {'Source': 'Config', 'Value': 1}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'MaxUp', {'Source': checkSource('MV', mv['MaxMove'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'MaxDn', {'Source': checkSource('MV', mv['MaxMove'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'MoveEnb', {'Source': 'Config', 'Value': 1}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'OptMode', {'Source': checkSource('MV', mv['OptMode'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'OptCoeff', {'Source': checkSource('MV', mv['OptCoeff'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'WindUp', {'Source': checkSource('MV', mv['WindUp'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'SST', {'Source': checkSource('MV', mv['SST'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'SP', {'Source': 'NotUsed'}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('close', 'MVs', 'Defaults', None, ConfigContent, ConfigParent)

for i, m in MV.items():
    MVConstant = []
    for j, x in mv.filter(items=[i+1], axis=0).to_dict().items():
        if type(x[i+1]) == int:
            MVConstant.append((j, x[i+1]))
    
    if len(MVConstant) == 0:
        ConfigContent, ConfigParent = appendTree('self-close', 'MVs', 'MV', {'Name': replaceName(m)}, ConfigContent, ConfigParent)
    else:
        ConfigContent, ConfigParent = appendTree('open', 'MVs', 'MV', {'Name': replaceName(m)}, ConfigContent, ConfigParent)
        for j, x in MVConstant:
            ConfigContent, ConfigParent = appendTree('self-close', 'MV', j, {'Value': x}, ConfigContent, ConfigParent)
        ConfigContent, ConfigParent = appendTree('close', 'MVs', 'MV', None, ConfigContent, ConfigParent)

ConfigContent, ConfigParent = appendTree('close', 'Configuration', 'MVs', None, ConfigContent, ConfigParent)

# DV
ConfigContent, ConfigParent = appendTree('open', 'Configuration', 'DVs', None, ConfigContent, ConfigParent)

ConfigContent, ConfigParent = appendTree('open', 'DVs', 'Defaults', None, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'Mode', {'Source': checkSource('DV', dv['Mode'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('close', 'DVs', 'Defaults', None, ConfigContent, ConfigParent)

for i, d in DV.items():
    DVConstant = []
    for j, x in dv.filter(items=[len(MV)+i+1], axis=0).to_dict().items():
        if type(x[len(MV)+i+1]) == int:
            DVConstant.append((j, x[len(MV)+i+1]))
    
    if len(DVConstant) == 0:
        ConfigContent, ConfigParent = appendTree('self-close', 'DVs', 'DV', {'Name': replaceName(d)}, ConfigContent, ConfigParent)
    else:
        ConfigContent, ConfigParent = appendTree('open', 'DVs', 'DV', {'Name': replaceName(d)}, ConfigContent, ConfigParent)
        for j, x in DVConstant:
            ConfigContent, ConfigParent = appendTree('self-close', 'DV', j, {'Value': x}, ConfigContent, ConfigParent)
        ConfigContent, ConfigParent = appendTree('close', 'DVs', 'DV', None, ConfigContent, ConfigParent)

ConfigContent, ConfigParent = appendTree('close', 'Configuration', 'DVs', None, ConfigContent, ConfigParent)

# CV
ConfigContent, ConfigParent = appendTree('open', 'Configuration', 'CVs', None, ConfigContent, ConfigParent)

ConfigContent, ConfigParent = appendTree('open', 'CVs', 'Defaults', None, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'Mode', {'Source': checkSource('CV', cv['Mode'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'High', {'Source': checkSource('CV', cv['High'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'Low', {'Source': checkSource('CV', cv['Low'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'HEnb', {'Source': 'Config', 'Value': 1}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'LEnb', {'Source': 'Config', 'Value': 1}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'OptMode', {'Source': checkSource('CV', cv['OptMode'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'OptCoeff', {'Source': checkSource('CV', cv['OptCoeff'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'PredErr', {'Source': 'ExternalPred'}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('self-close', 'Defaults', 'SST', {'Source': checkSource('CV', cv['SST'])}, ConfigContent, ConfigParent)
ConfigContent, ConfigParent = appendTree('close', 'CVs', 'Defaults', None, ConfigContent, ConfigParent)

for i, c in CV.items():
    CVConstant = []
    for j, x in cv.filter(items=[len(MV)+len(DV)+i+1], axis=0).to_dict().items():
        if type(x[len(MV)+len(DV)+i+1]) == int:
            CVConstant.append((j, x[len(MV)+len(DV)+i+1]))
    
    if len(CVConstant) == 0:
        ConfigContent, ConfigParent = appendTree('self-close', 'CVs', 'CV', {'Name': replaceName(c)}, ConfigContent, ConfigParent)
    else:
        ConfigContent, ConfigParent = appendTree('open', 'CVs', 'CV', {'Name': replaceName(c)}, ConfigContent, ConfigParent)
        for j, x in CVConstant:
            ConfigContent, ConfigParent = appendTree('self-close', 'CV', j, {'Value': x}, ConfigContent, ConfigParent)
        ConfigContent, ConfigParent = appendTree('close', 'CVs', 'CV', None, ConfigContent, ConfigParent)

ConfigContent, ConfigParent = appendTree('close', 'Configuration', 'CVs', None, ConfigContent, ConfigParent)

ConfigContent, ConfigParent = appendTree('close', None, 'Configuration', None, ConfigContent, ConfigParent)

print(ConfigContent)

with open(CONFIG_PATH, 'w') as f:
    f.write(ConfigContent)

<?xml version="1.0" encoding="utf-8"?>
<Configuration xmlns="http://schemas.matrikon.com/2010/08/CPM/GenericAPC.xsd">
	<Controller Name="GSP5_DEE">
		<ExecutionInterval Value="60" />
		<HP Value="180" />
		<Mode Source="External" />
		<OptMode Source="NotUsed" />
		<OptCost Source="NotUsed" />
	</Controller>
	<MVs>
		<Defaults>
			<Mode Source="External" />
			<High Source="External" />
			<Low Source="External" />
			<HEnb Source="Config" Value="1" />
			<LEnb Source="Config" Value="1" />
			<MaxUp Source="External" />
			<MaxDn Source="External" />
			<MoveEnb Source="Config" Value="1" />
			<OptMode Source="NotUsed" />
			<OptCoeff Source="NotUsed" />
			<WindUp Source="NotUsed" />
			<SST Source="External" />
			<SP Source="NotUsed" />
		</Defaults>
		<MV Name="3503FICA012_SV" />
		<MV Name="3503FICA016_SV" />
		<MV Name="3503LICA016_MV" />
		<MV Name="3503PICA032_SV" />
		<MV Name="3503TICA058_SV" />
		<MV Name="3511FICA004_SV" />
		<MV Name="3519FICA002_SV" />
	</MVs>
	<DVs>
		<D